In [1]:
import pandas as pd
import unicodedata

In [ ]:
# マスター
# df = pd.read_csv('master.csv',
#                  usecols=[1, 4, 6, 7, 8, 9, 26, 27, 29, 45],
#                  dtype = str).fillna(0).astype({'収容数': int, '回転枚数': int, '読取枚数': int, '発注枚数': int})
# df.rename(columns={'かんＳＥＬＦ': 'ad',
#                     '仕入先': 'sup_code',
#                     '背番号': 'seban',
#                     '品番': 'hinban',
#                     '収容数': 'num',
#                     'ストアアドレス': 'store',
#                     '回転枚数': 'k_num',
#                     '読取枚数': 'y_num',
#                     '発注枚数': 'h_num',
#                     '箱種': 'box'}, inplace=True)

# df['seban'] = df['seban'].str.strip()
# df['hinban'] = df['hinban'].str.strip()
# df['store'] = df['store'].str.strip()
# df['box'] = df['box'].str.strip()
# df.head()

In [2]:
def add_space_if_fw(text):
    if unicodedata.east_asian_width(text) in 'FWA':
        return text + ' '
    else:
        return text

In [27]:
file_name = f'USROUT'

with open(file_name, 'r', encoding='cp932') as f:
    content = f.read()
result = ''

for ch in content:
    result += add_space_if_fw(ch)

file_name_changed = f'changed_USROUT.csv'

with open(file_name_changed, mode='w', encoding='UTF-8') as f:
    f.write(result)

widths = [1, 5, 5, 20]
names = ['aki', 'add', 'sup_code', 'sup_name']
usecols= [2, 3]

df = pd.read_fwf(file_name_changed, widths=widths, names=names, usecols=usecols, encoding='UTF-8', dtype = str).fillna(0)
df['sup_name'] = df['sup_name'].str.strip()


df.tail()

,sup_code,sup_name
606,99980,工 程 内 Ｂ Ｗ ０ １
607,99990,工 程 内 Ｓ Ｗ １ ８
608,99998,暫 定
609,99999,誤 品 照 合 テ ス ト 用
610,0,NaN


In [15]:
import sqlite3

In [16]:
con = sqlite3.connect('test.db')
cur = con.cursor()

In [12]:
# テーブル削除
cur.execute(
    'DROP TABLE IF EXISTS master'
)

In [17]:
# マスター作成
cur.execute(
    '''
    CREATE TABLE master (
        id INTEGER PRIMARY KEY, 
        ad TEXT, 
        sup_code TEXT, 
        seban TEXT, 
        hinban TEXT, 
        num INTEGER, 
        store TEXT, 
        k_num INTEGER, 
        y_num INTEGER, 
        h_num INTEGER,
        box TEXT)
    '''
)						

In [56]:
# 集欠マスター作成
cur.execute(
    '''
    CREATE TABLE shuketu (
        id INTEGER PRIMARY KEY, 
        ad TEXT, 
        num INTEGER,
        num_all INTEGER, 
        cust_name TEXT, 
        due_date TEXT, 
        tonyu INTEGER,
        inventory INTEGER, 
        afure INTEGER, 
        shuketubi TEXT, 
        bin INTEGER,
        comment TEXT)
    '''
)		

In [6]:
# 仕入先マスター作成
cur.execute(
    '''
    CREATE TABLE sup (
        id INTEGER PRIMARY KEY, 
        sup_code TEXT,
        sup_name TEXT)
    '''
)		

In [18]:
# マスター→DB
df.to_sql('master', con, if_exists='append', index=False)

40048

In [9]:
# 仕入先マスター→DB
df_s.to_sql('sup', con, if_exists='append', index=False)

596

In [ ]:
cur.execute(
    "select * from sqlite_master where type='table' and name='master'"
)
tables = cur.fetchall()
tables

In [4]:
# 全テーブル検索
cur.execute(
    "SELECT NAME FROM sqlite_master WHERE type='table'"
)
tables = cur.fetchall()
tables

[('sqlite_sequence',), ('shuketu',), ('sup',), ('master',)]

In [ ]:
cur.execute(
    "SELECT * FROM master WHERE hinban like '82715%'"
)
tables = cur.fetchall()
tables

In [ ]:
cur.execute(
    '''
    INSERT INTO
	    shuketu
    VALUES
        (1,
        '00011',
        1,
        10,
        'aaa',
        '',
        2,
        1,
        1,
        date('now', 'localtime'),
        1,
        '')
    '''
)

In [6]:
con.close()

In [7]:
# モデル設定
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm import Session
import datetime
from sqlalchemy import Column, ForeignKey, Integer, String, ForeignKey
from sqlalchemy.orm import relationship

SQLALCHEMY_DATABASE_URL = 'sqlite:///test.db'


engine = create_engine(
  SQLALCHEMY_DATABASE_URL,
  connect_args={'check_same_thread': False}
)

SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

class Master(Base):
  __tablename__ = 'master'
  id = Column(Integer, primary_key=True, autoincrement=True, index=True)
  ad = Column(String, unique=True, index=True)
  sup_code = Column(String, index=True)
  seban = Column(String, index=True)
  hinban = Column(String, index=True)
  num = Column(Integer, index=True)
  store = Column(String, index=True)
  k_num = Column(Integer, index=True)
  y_num = Column(Integer, index=True)
  h_num = Column(Integer, index=True)
  shuketu = relationship('Shuketu', backref="master")

class Shuketu(Base):
  __tablename__ = 'shuketu'
  id = Column(Integer, primary_key=True, autoincrement=True, index=True)
  ad = Column(String, ForeignKey('master.ad'), index=True)
  num = Column(Integer, index=True)
  num_all = Column(Integer, index=True)
  cust_name = Column(String, index=True)
  due_date = Column(String, index=True)
  tonyu = Column(Integer, index=True)
  inventory = Column(Integer, index=True)
  afure = Column(Integer, index=True)
  shuketubi = Column(String, index=True)
  bin = Column(Integer, index=True)
  comment = Column(String, index=True)
  

In [8]:
# create
# def create_item(db, args):
#     item = Master(
#         ad=args['ad'], 
#         sup_code=args['sup_code'], 
#         seban=args['seban'], 
#         hinban=args['hinban'], 
#         num=args['num'], 
#         store=args['store'], 
#         k_num=args['k_num'], 
#         y_num=args['y_num'], 
#         h_num=args['h_num'])
#     db.add(item)
#     db.commit()
#     db.refresh(item)
#     return item

def create_item(db, args):

    item = Shuketu(
        ad=args['ad'],
        num=args['num'],
        num_all=args['num_all'],
        cust_name=args['cust_name'],
        due_date=args['due_date'],
        tonyu=args['tonyu'],
        inventory=args['inventory'],
        afure=args['afure'],
        shuketubi=args['shuketubi'],
        bin=args['bin'],
        comment=args['comment'],
    )
    db.add(item)
    db.commit()
    db.refresh(item)
    return item


# args = {}
# args['ad'] = '00001'
# args['sup_code'] = '00100'
# args['seban'] = '010'
# args['hinban'] = '12345-67890-00'
# args['num'] = 100
# args['store'] = 'A01-01-1'
# args['k_num'] = 1
# args['y_num'] = 1 
# args['h_num'] = 1

args = {}
args['ad']= '00010'
args['num']= 1
args['num_all']= 10
args['cust_name']= 'aaa'
args['due_date']= ''
args['tonyu']= 2
args['inventory']= 1
args['afure']= 1
args['shuketubi']= datetime.date.today().isoformat()
args['bin']= 1
args['comment']= ''


db = SessionLocal()
item = create_item(db, args)    
# print(item.id, item.ad, item.sup_code, item.seban, item.hinban, item.num, item.store, item.k_num, item.y_num, item.h_num)
db.close()

In [8]:
# read
# def read_item(db, ad):
#   items = db.query(Master).filter(Master.ad == ad).all()
#   return items

def read_item(db, ad):
  items = db.query(Shuketu).filter(Shuketu.ad == ad).all()
  return items


ad = '00010'
db = SessionLocal()
items = read_item(db, ad)
for item in items:
    # print(item.id, item.ad, item.sup_code, item.seban, item.hinban, item.num, item.store, item.k_num, item.y_num, item.h_num)
    print(item.id, item.ad, item.num, item.num_all, item.cust_name, item.due_date, item.tonyu, item.inventory, item.afure, item.shuketubi, item.bin, item.comment)
    print(item.master.hinban)
db.close()

4 00010 1 10 とよた  2 1 1 2022-05-18 1 
PSBBL-20000-00


In [13]:
# update
# def update_item(db, args):
#     item = db.query(Master).filter(Master.ad == args['ad']).first()
#     item.num = args['num']
#     db.commit()
#     db.refresh(item)
#     return item

def update_item(db, args):
    item = db.query(Shuketu).filter(Shuketu.id == args['id']).first()
    item.num = args['num']
    db.commit()
    db.refresh(item)
    return item

args = {}
args['id'] = 3
args['ad'] = '00001'
args['sup_code'] = '00100'
args['seban'] = '010'
args['hinban'] = '12345-67890-00'
args['num'] = 50000
args['store'] = 'A01-01-1'
args['k_num'] = 1
args['y_num'] = 1 
args['h_num'] = 1

db = SessionLocal()
item = update_item(db, args)
# print(item.id, item.ad, item.sup_code, item.seban, item.hinban, item.num, item.store, item.k_num, item.y_num, item.h_num)
print(item.num)
db.close()

50000


In [15]:
# delete
# def delete_item(db, id):
#     item = db.query(Master).filter(Master.id == id).first()
#     db.delete(item)
#     db.commit()

def delete_item(db, id):
    item = db.query(Shuketu).filter(Shuketu.id == id).first()
    db.delete(item)
    db.commit()

id = 3

db = SessionLocal()
delete_item(db, id)
db.close()